In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import gensim
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer


# Reading the data

In [ ]:
dldata=pd.read_csv("../input/disneyland/DisneylandReviews.csv",encoding='latin-1')

In [ ]:
dldata

In [ ]:
dldata.shape

In [ ]:
dldata=pd.read_csv("../input/disneyland/DisneylandReviews.csv",encoding='latin-1',na_values="missing")

In [ ]:
dldata

# Checking for null values¶

In [ ]:
dldata.isnull().sum()

In [ ]:
dldata.Review_ID.nunique()

In [ ]:
dldata.Review_ID.value_counts()

# Removing Duplicate

In [ ]:
dldata.drop_duplicates(subset="Review_ID",inplace=True,keep='first')

In [ ]:
dldata

# visualizing data

Branch

In [ ]:
dldata.Branch.nunique()

In [ ]:
dldata.Branch.value_counts()

In [ ]:
sns.set_style("whitegrid")
sns.countplot(x="Branch", data=dldata)

Reviewer Location

In [ ]:
dldata.Reviewer_Location.nunique()

In [ ]:
dldata.Reviewer_Location.value_counts().head()

Rating

In [ ]:
dldata.Rating.value_counts()

In [ ]:
dldata.Rating.nunique()

In [ ]:
sns.set_style("whitegrid")
sns.countplot(x='Rating',data=dldata)

In [ ]:
sns.set_style("whitegrid")
sns.countplot(x='Rating',hue='Branch',data=dldata)

In [ ]:
Reviewer_Locations = dldata['Reviewer_Location'].value_counts()
rl_list = list(Reviewer_Locations.index)

In [ ]:
rl_list

In [ ]:
plt.figure(figsize=(10, 20))

figure = plt.pie(Reviewer_Locations.sort_values(ascending=False).iloc[:10],pctdistance=0.6,labeldistance=1.1,
                 colors='rgb',labels=rl_list[:10],autopct='%1.2f%%')

In [ ]:
def npn_rating(rating):
    if rating <3:
        return -1 #Negtive Review
    elif rating >3:
        return 1 #Positive Review
    else:
        return 0 #neutral Review
    
dldata['Rating'] = dldata['Rating'].apply(lambda rating: npn_rating(rating))

In [ ]:
# get the independent feature
x=dldata.drop('Rating',axis=1)

In [ ]:
x.head()

In [ ]:
# get the dependent feature
y=dldata['Rating']

In [ ]:
y.head()

In [ ]:
# Droping the null values
dldata=dldata.dropna()
dldata.head()

In [ ]:
messages=dldata.copy()

In [ ]:
# reset the index
messages.reset_index(inplace=True)

In [ ]:
messages.shape

In [ ]:
messages['Review_Text'][6]

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['Review_Text'][i]) #cleaning the data
    review = review.lower() # converting into lowercase
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #using stop-words
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[6]

In [ ]:
## TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,3)) #getting top 5000 features
X=tfidf_v.fit_transform(corpus).toarray()

In [ ]:
X.shape

In [ ]:
y=messages['Rating']

In [ ]:
y.head(5)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
# top 20 most frequest occurring words
tfidf_v.get_feature_names()[:20]

In [ ]:
# Detail explanation
tfidf_v.get_params()

In [ ]:
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())

In [ ]:
count_df.head()

# MultinomialNB Algorithm

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()

In [ ]:
from sklearn import metrics
import itertools

In [ ]:
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
score